### 🏦 Proyecto: Predicción de Churn en Beta Bank

#### 📌 Descripción general

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

#### 🎯 Objetivo

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 




### 📂 Descripción de los datos

#### 🗂️ Fuente de datos

- Archivo: `/datasets/Churn.csv`

#### 🔢 Características (features)

- `RowNumber`: índice de cadena de datos (número de fila)
- `CustomerId`: identificador único del cliente
- `Surname`: apellido del cliente
- `CreditScore`: Score de crédito del cliente
- `Geography`: país de residencia del cliente
- `Gender`: sexo del cliente
- `Age`: edad del cliente
- `Tenure`: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- `Balance`: saldo de la cuenta bancaria
- `NumOfProducts`: número de productos bancarios utilizados por el cliente
- `HasCrCard`: el cliente tiene una tarjeta de crédito  
  - `1` - sí  
  - `0` - no
- `IsActiveMember`: actividad del cliente  
  - `1` - cliente activo  
  - `0` - cliente inactivo
- `EstimatedSalary`: salario estimado del cliente


#### 🎯 Variable objetivo

- `Exited`: indica si el cliente **se ha ido** del banco  
  - `1` - el cliente se fue  
  - `0` - el cliente se quedó


### 0. Estrategia inicial

El objetivo principal es identificar a los clientes que tienen alta probabilidad de **abandonar el banco**.  
Para ello, vamos a entrenar **modelos de clasificación** que aprendan a distinguir entre:

- Clientes que **se quedaron** (`Exited = 0`).
- Clientes que **se fueron** (`Exited = 1`).

Primero analizamos a los clientes que ya se han ido para entender **qué características están asociadas al churn**.  
Después, utilizamos ese conocimiento para predecir, entre los clientes actuales, **quiénes tienen mayor riesgo de irse** y así poder implementar **estrategias de retención** (por ejemplo, ofertas personalizadas, contacto proactivo o revisión de condiciones del producto).


### 1. 📦 Importación de librerías

In [50]:
# 1.1 Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler 

from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    f1_score,
    roc_auc_score,
    recall_score,
    average_precision_score,
    precision_recall_curve
)

# 1.2  Definimos el random State para toda nuestra exploración de datos.
RANDOM_STATE = 54321


### 📥 2. Carga de datos



#### 📄 2.1 Lectura del archivo

In [ ]:
# Ruta de donde se encuntra nuetra base de datos


from pathlib import Path

BASE_DIR = Path.cwd()

RUTA = BASE_DIR / "Churn.csv"

df = pd.read_csv(RUTA)
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


#### 👀 2.2 Vista general del DataFrame

En esta sección revisamos:

- Tipos de datos.
- Presencia de valores nulos.
- Tamaño del dataset.

Los datos que tenemos en nuestro DataFrame parecen estar bien, lo que me nos puede resualtar un poco inquietante son los valores nulos.

Por otro lado tendremos que quitar `RowNumer` ya que podemos trabajar con el indice, nuesta,  columna objetivo `Exited` esta en **float** por lo que tenemos que cambiarla a int porque no tiene decimales.

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### 🔍 3. Análisis exploratorio inicial

#### 3.1 Revisión de valores nulos en `Tenure`

La columna `Tenure` tiene valores nulos. Queremos entender:

- ¿Cuántos faltan?
- ¿Qué porcentaje representan?
- ¿Si hay algún patrón en esos registros?


Los datos nulos tenemos 909 que es un 9.09% de los datos

In [53]:
# Contamos los valores nuelos de la columna "Tenure"
tenure_na_count = df["Tenure"].isna().sum()
# Sacamos el porcentaje de los valores nuelos de la columna "Tenure"
tenure_na_ratio = df["Tenure"].isna().mean() * 100

# Imprimimos los resultados
print("Valores nulos en Tenure:", tenure_na_count)
print("Porcentaje de nulos en Tenure: {:.2f}%".format(tenure_na_ratio))

# Filtamos los valores nulos para ver si hya alguna similitud de manera visual
df_null = df[df["Tenure"].isna()]
# Filtamos los valores 0 para ver si hya alguna similitud con los valores nuelos
df_zero = df[df["Tenure"] == 1]

df_null.head()


Valores nulos en Tenure: 909
Porcentaje de nulos en Tenure: 9.09%


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0


In [54]:
# Mostramos los peremos 10 de las personas que se han ido del banco.
df_zero.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
16,17,15737452,Romeo,653,Germany,Male,58,1.0,132602.88,1,1,0,5097.67,1
54,55,15569590,Yoo,601,Germany,Male,42,1.0,98495.72,1,1,0,40014.76,1
55,56,15760861,Phillipps,619,France,Male,43,1.0,125211.92,1,1,1,113410.49,0
62,63,15702014,Jeffrey,555,Spain,Male,33,1.0,56084.69,2,0,0,178798.13,0
66,67,15696061,Brownless,581,Germany,Female,34,1.0,101633.04,1,1,0,110431.51,0
75,76,15780961,Cavenagh,735,France,Female,21,1.0,178718.19,2,1,0,22388.00,0
80,81,15706021,Buley,665,France,Female,34,1.0,96645.54,2,0,0,171413.66,0
104,105,15804919,Dunbabin,670,Spain,Female,65,1.0,0.00,1,1,1,177655.68,1


No encontramos ningun posible patron visual que podamos crear una hipotesis por lo que mas adelate dividiremos los valores nulos para ver si estan relacionados con posible churn.

#### 📊 3.2 Estadísticos descriptivos

Revisamos la distribución de las variables numéricas para detectar rangos, posibles valores atípicos y entender el comportamiento general de los clientes y formular posibles hipotesis de los datos.


In [55]:
df.describe()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Tendremos que escalar las columnas `CreditScore`, `Balance`, `EstimatedSalary` par  que todas las columnas esten en la misma escala.

#### 3.3 Duplicados

Comprobamos si hay filas duplicadas en el dataset, tanto a nivel de fila completa como por `CustomerId`.


In [56]:
print(df.duplicated().sum())
print(df.duplicated("CustomerId").sum())



0
0


No se encontraron ningun valor duplicado por lo que podemos proceder a la preparacion de los datos.

### 🧹 4. Preparación y preprocesamiento de datos

#### 4.1 Manejo de valores faltantes en `Tenure`

Suposición de trabajo:
- `Tenure` representa la antigüedad del cliente.
- No encontramos evidencia clara de que los nulos correspondan solo a clientes nuevos.
- Para no perder registros, remplasamos con la **mediana** y marcamos qué registros tenían nulo originalmente.

Creamos la columna binaria `TenureWasMissing` para capturar ese posible efecto siguiendo la nomenclatura de las columnas.




In [ ]:

# Marcamos dónde faltaba Tenure
df["TenureWasMissing"] = df["Tenure"].isna().astype(int)

# Calculamos la mediana solo con valores existentes
tenure_median = df["Tenure"].median()

# Rempalzamos Tenure con la mediana
df["Tenure"] = df["Tenure"].fillna(tenure_median)

# Mostramos las primeras filas
df[["Tenure", "TenureWasMissing"]].head()


,Tenure,TenureWasMissing
0,2.0,0
1,1.0,0
2,8.0,0
3,1.0,0
4,2.0,0


#### 4.2 Ajuste de tipos de datos

Convertimos las columnas numéricas a los tipos adecuados (`int` o `float`) para evitar problemas en el modelado.


In [ ]:
# Enlistamos los tipos de datos pro clolumna (Feature) 
df.dtypes


RowNumber             int64
CustomerId            int64
Surname              object
CreditScore           int64
Geography            object
Gender               object
Age                   int64
Tenure              float64
Balance             float64
NumOfProducts         int64
HasCrCard             int64
IsActiveMember        int64
EstimatedSalary     float64
Exited                int64
TenureWasMissing      int64
dtype: object

In [ ]:
# Seleccionamos las caracteristicas que queremos convertir a float 
col_float = ["Balance", 
             "EstimatedSalary"
             ]

# Seleccionamos las caracteristicas que queremos convertir a int
# algunos datos ya se encontraban en ese tipo de dato aun asi los incluimos para estar seguros

col_int = [
    "RowNumber",
    "CreditScore",
    "Age",
    "Tenure",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "Exited"
]

# Convertimos las caracteristicas al tipo de dato seleccionado anterior mente
df[col_float] = df[col_float].astype(float)
df[col_int] = df[col_int].astype(int)

df.dtypes


RowNumber             int64
CustomerId            int64
Surname              object
CreditScore           int64
Geography            object
Gender               object
Age                   int64
Tenure                int64
Balance             float64
NumOfProducts         int64
HasCrCard             int64
IsActiveMember        int64
EstimatedSalary     float64
Exited                int64
TenureWasMissing      int64
dtype: object

#### 4.3 Codificación de variables categóricas (One-Hot Encoding)

Variables categóricas a codificar:

- `Geography`
- `Gender`

Usamos **One-Hot Encoding** con `drop_first=True` para evitar multicolinealidad perfecta.


In [ ]:
# seleccionamos las columnas las cuales queremos aplicarle OHE
col_ohe = ["Geography", "Gender"]

# Quitamos una de las columnas para evitar multicolinealidad perfecta
dummies = pd.get_dummies(df[col_ohe], drop_first=True)

# Quitamos las columnas originales para evitar duplicidad de datos
df_num = df.drop(columns=col_ohe)

# Unimos las nuevas columnas a unetro data set
df_prepared = pd.concat([df_num, dummies], axis=1)

# comprobamos como quedo nuestro data set despues de la codificacion OHE
df_prepared.head()


,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,TenureWasMissing,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,0,False,False,False
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,False,True,False
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,0,False,False,False
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,0,False,False,False
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,False,True,False


### ⚖️ 5. Análisis del equilibrio de clases

Revisamos la distribución de la variable objetivo `Exited`:

- `1` → el cliente se fue.
- `0` → el cliente se quedó.

Esto nos indica si el problema está desbalanceado.


In [ ]:
# Seleccionamos como la variable objetivo está distribuida
target_counts = df_prepared["Exited"].value_counts()
# Sacamos los porcentajes de la distribucion de la variable objetivo
target_ratio = df_prepared["Exited"].value_counts(normalize=True) * 100

# Revisamos los resultados
print(target_counts)
print("\nPorcentaje por clase (%):")
print(target_ratio.round(2))


Exited
0    7963
1    2037
Name: count, dtype: int64

Porcentaje por clase (%):
Exited
0    79.63
1    20.37
Name: proportion, dtype: float64


Esto muestra que aproximadamente 4 de cada 5 clientes se quedan y solo 1 de cada 5 se va, por lo que el problema es claramente desbalanceado a favor de la clase 0.

Debido a este desbalance, más adelante apliqué upsampling sobre el conjunto de entrenamiento para aumentar la proporción de la clase minoritaria (Exited = 1) y ayudar al modelo a aprender mejor a detectar a los clientes que abandonan el banco.

### 📊 6. División de los datos en conjuntos

Usamos un split Train/Valid:

- **Train (75%)** → entrenamiento y balanceo.
- **Valid (25%)** → evaluación de modelos y tuning de hiperparámetros.

Escalamos las columnas numéricas continuas `CreditScore`, `Balance` y `EstimatedSalary` usando `StandardScaler`.

In [62]:
# Seleccionamos las columnas que pueden causar problemas en nuestros modelos. 
col_drop = ["RowNumber", "Surname", "Exited", "CustomerId"]

# Quitamos las columnas.
features = df_prepared.drop(columns=col_drop, axis=1)
target = df_prepared["Exited"]

# Dividomos nuestro modelo con 75% para el test y 25 % para la validación.
features_train, features_valid, target_train, target_valid = train_test_split(
    features,
    target,
    test_size=0.25,
    random_state=RANDOM_STATE
)

# Imprimimos el tamaño de cada conjunto para ver que la división se halla realizado con éxito.
print("features_train:", features_train.shape)
print("features_valid:", features_valid.shape)
print("target_train:", target_train.shape)
print("target_valid:", target_valid.shape)


# ==========================
# Escalado de características
# ==========================

# Columnas numéricas continuas a escalar.
# No se incluyen las otras ya que se encuentan en una escala similar.
cols_to_scale = [
    "CreditScore",
    "Balance",
    "EstimatedSalary"
]

scaler = StandardScaler()

# Ajustamos el scaler SOLO con los datos de entrenamiento
features_train_scaled = features_train.copy()
features_valid_scaled = features_valid.copy()

features_train_scaled[cols_to_scale] = scaler.fit_transform(features_train[cols_to_scale])
features_valid_scaled[cols_to_scale] = scaler.transform(features_valid[cols_to_scale])

# Imprimimos para verificar como quedo el escalado de los datos. 
features_train_scaled.head()



features_train: (7500, 12)
features_valid: (2500, 12)
target_train: (7500,)
target_valid: (2500,)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,TenureWasMissing,Geography_Germany,Geography_Spain,Gender_Male
7770,0.507706,29,3,0.779438,1,1,1,0.896894,0,False,False,True
3552,1.890945,39,3,-1.234086,2,1,0,-1.716892,0,False,False,False
2141,0.249639,18,7,-1.234086,2,1,1,-0.774646,0,False,False,True
8832,1.065131,35,4,-1.234086,2,1,1,0.106717,0,False,False,True
9693,-0.338753,51,10,1.445615,1,0,0,-0.117565,0,True,False,False


### ⚖️ 7. Balanceo de clases (Upsampling)

El conjunto de entrenamiento está desbalanceado.  
Para que el modelo aprenda mejor la clase minoritaria (`Exited = 1`), usamos **upsampling** sobre el conjunto de entrenamiento:

- Repetimos varias veces las filas de la clase minoritaria.
- Mezclamos (`shuffle`) para no introducir orden artificial.


In [ ]:
# 7.1 Funcion para hacer sobre muestreo de las varibles

# Definimos las variables que necesitamos dentro de los () 
def upsample(features, target, repeat, random_state=RANDOM_STATE):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
# Juntamos los datos sobremuestrados 
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

# Aplicamos aleatoridad a nuestros datos
    features_upsampled, target_upsampled = shuffle(
        features_upsampled,
        target_upsampled,
        random_state=random_state
    )
    return features_upsampled, target_upsampled

# Multiplicamos por 3 para acercanos lo mayor posinble a el tamaño de nuestra variable dominante.
repeat = 3
features_upsampled, target_upsampled = upsample(
    features_train_scaled,
    target_train,
    repeat
)

# Revisamos los resultados
print("Tamaño original (train):", target_train.shape[0])
print("Tamaño tras upsampling:", target_upsampled.shape[0])
print("\nDistribución tras upsampling:")
print(target_upsampled.value_counts())


Tamaño original (train): 7500
Tamaño tras upsampling: 10618

Distribución tras upsampling:
Exited
0    5941
1    4677
Name: count, dtype: int64


A partir de aquí, en TODO lo que sigue, usa:

`features_upsampled` / `target_upsampled` para entrenar.

`features_valid_scaled` / `target_valid` para validar.

---

### 🧪 8. Modelo base (sin tuning) – DecisionTreeClassifier

Primero entrenamos un **modelo base simple** con `DecisionTreeClassifier`  
usando directamente el conjunto de entrenamiento original (sin balanceo) para tener una referencia.


In [ ]:
# 8.1 entrenamos nuestro modelo base
tree_base = DecisionTreeClassifier(
    random_state=RANDOM_STATE,
    max_depth=2
)

# Se usara el train normal (escalado), sin upsampling.
tree_base.fit(features_train_scaled, target_train)
base_tree_pred = tree_base.predict(features_valid_scaled)

print("F1 modelo base (árbol simple):", f1_score(target_valid, base_tree_pred))

# AUC-ROC (si lo dejas así, sigue usando etiquetas; ver punto 2 para mejorarlo)
print("AUC-ROC (basado en predicción binaria):", roc_auc_score(target_valid, base_tree_pred))



F1 modelo base (árbol simple): 0.5177304964539007
AUC-ROC (basado en predicción binaria): 0.6922347896982564


### ⚙️ 9. Búsqueda de hiperparámetros

Probamos distintos modelos y configuraciones usando el conjunto **upsampled**:

- `DecisionTreeClassifier`
- `RandomForestClassifier`
- `LogisticRegression`

Buscamos maximizar **F1** sobre el conjunto de validación.


In [65]:
# 9.1 Buscando mejores híper parámetros para DecisionTreeClassifier 

best_f1_tree = 0
best_leafs = 0
best_depth_tree = 0

for leafs in range(10, 200, 10):
    for depth in range(1, 11):
        model_dtc = DecisionTreeClassifier(
            min_samples_leaf=leafs,
            max_depth=depth,
            random_state=RANDOM_STATE
        )
        
        model_dtc.fit(features_upsampled, target_upsampled)
        predictions_valid = model_dtc.predict(features_valid_scaled)
        f1 = f1_score(target_valid, predictions_valid)
        # Imprime cada uno de los parametros provados.
        # 👇 Para ver solo el resultado final borrar esta linea.
        print("leafs=", leafs, ", depth=", depth, "is", f1)

        # Revisamos que cada valor de f1 se mayor al que tenemos guardado.
        if f1 > best_f1_tree:
            best_f1_tree = f1
            best_leafs = leafs
            best_depth_tree = depth

# Imprimimos el mejor resultado encontrado con nuestros híper parametros.
print(
    "Mejor DecisionTreeClassifier - F1:",
    best_f1_tree,
    "| min_samples_leaf:", best_leafs,
    "| max_depth:", best_depth_tree
)


leafs= 10 , depth= 1 is 0.47959183673469385
leafs= 10 , depth= 2 is 0.5126945126945127
leafs= 10 , depth= 3 is 0.5301837270341208
leafs= 10 , depth= 4 is 0.5228276877761414
leafs= 10 , depth= 5 is 0.5834076717216771
leafs= 10 , depth= 6 is 0.5595854922279793
leafs= 10 , depth= 7 is 0.5789923142613151
leafs= 10 , depth= 8 is 0.5977653631284916
leafs= 10 , depth= 9 is 0.5437710437710438
leafs= 10 , depth= 10 is 0.5563380281690141
leafs= 20 , depth= 1 is 0.47959183673469385
leafs= 20 , depth= 2 is 0.5126945126945127
leafs= 20 , depth= 3 is 0.5301837270341208
leafs= 20 , depth= 4 is 0.5228276877761414
leafs= 20 , depth= 5 is 0.585278276481149
leafs= 20 , depth= 6 is 0.5597624350408315
leafs= 20 , depth= 7 is 0.5914260717410323
leafs= 20 , depth= 8 is 0.5956678700361011
leafs= 20 , depth= 9 is 0.5468227424749164
leafs= 20 , depth= 10 is 0.5559322033898305
leafs= 30 , depth= 1 is 0.47959183673469385
leafs= 30 , depth= 2 is 0.5126945126945127
leafs= 30 , depth= 3 is 0.5301837270341208
leafs= 

Mejor DecisionTreeClassifier - F1: 0.5982758620689655 | min_samples_leaf: 70 | max_depth: 7


In [66]:
# 9.2 Buscando mejores híper parámetros para RandomForestClassifier 

best_f1_rf = 0
best_estimators_rf = 0
best_depth_rf = 0

for est in range(10, 101, 10):
    for depth in range(10, 61, 10):
        model = RandomForestClassifier(
            random_state=RANDOM_STATE,
            n_estimators=est,
            max_depth=depth
        )
        
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid_scaled)
        f1 = f1_score(target_valid, predictions_valid)
        # Imprime cada uno de los parametros provados.
        # 👇 Para ver solo el resultado final borrar esta linea.
        print("n_estimators", est, ", depth=", depth, "f1", f1)
       
        # Revisamos que cada valor de f1 se mayor al que tenemos guardado.
        if f1 > best_f1_rf:
            best_f1_rf = f1 
            best_estimators_rf = est
            best_depth_rf = depth


# Imprimimos el mejor resultado encontrado con nuestros híper parametros.
print(
    "Mejor RandomForestClassifier - F1:",
    best_f1_rf,
    "| n_estimators:", best_estimators_rf,
    "| max_depth:", best_depth_rf
)


n_estimators 10 , depth= 10 f1 0.6109452736318408
n_estimators 10 , depth= 20 f1 0.5727810650887574
n_estimators 10 , depth= 30 f1 0.5565819861431871
n_estimators 10 , depth= 40 f1 0.5565819861431871
n_estimators 10 , depth= 50 f1 0.5565819861431871
n_estimators 10 , depth= 60 f1 0.5565819861431871
n_estimators 20 , depth= 10 f1 0.6007984031936128
n_estimators 20 , depth= 20 f1 0.584971098265896
n_estimators 20 , depth= 30 f1 0.5885057471264368
n_estimators 20 , depth= 40 f1 0.5885057471264368
n_estimators 20 , depth= 50 f1 0.5885057471264368
n_estimators 20 , depth= 60 f1 0.5885057471264368
n_estimators 30 , depth= 10 f1 0.6115035317860746
n_estimators 30 , depth= 20 f1 0.5825688073394495
n_estimators 30 , depth= 30 f1 0.5829493087557603
n_estimators 30 , depth= 40 f1 0.5829493087557603
n_estimators 30 , depth= 50 f1 0.5829493087557603
n_estimators 30 , depth= 60 f1 0.5829493087557603
n_estimators 40 , depth= 10 f1 0.6130653266331658
n_estimators 40 , depth= 20 f1 0.5965317919075145
n

Mejor RandomForestClassifier - F1: 0.6212424849699398 | n_estimators: 100 | max_depth: 10



#### 10. 📈 Métricas 



In [ ]:
# 10.1 Función para obtención de las métricas de cada uno de nuestros modelos.

def evaluar_modelo(model, X_valid, y_valid, nombre_modelo="modelo"):
    
    # Predicciones de clase
    y_pred = model.predict(X_valid)
    
    # Porvamos la probabilidades de la clase positiva (si el modelo las soporta)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_valid)[:, 1]
    else:
        print("error")

    # Matriz de confución.
    tn, fp, fn, tp = confusion_matrix(y_valid, y_pred).ravel()
    # Metricas relebantes a los modelos de Clasifición.
    recall = recall_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred)
    f1 = f1_score(y_valid, y_pred)
    auc_roc = roc_auc_score(y_valid, y_proba)
    curva_pr = average_precision_score(y_valid, y_proba)

    # Pasamos los datos a un lista para ponerlo en una tabla
    metrics = pd.DataFrame({
        "modelo": [nombre_modelo],
        "Recall": [recall],
        "F1": [f1],
        "Curva PR": [curva_pr],
        "AUC-ROC": [auc_roc], 
        "Precision": [precision],
        "TP": [tp],
        "FP": [fp],
        "TN": [tn],
        "FN": [fn],

    })
    return metrics


In [68]:
resultados = []

#=========================
# Baseline Modelo (base) 
#=========================

# Modelo (base) 
tree_base = DecisionTreeClassifier(
    random_state=RANDOM_STATE, # Definido en el punto 1.2
    max_depth=2 # Se utilizo 2 como punto de partida del modelo base.
)

# Se usara el train normal (escalado), sin upsampling.
tree_base.fit(features_train_scaled, target_train)

metrics_baseline = evaluar_modelo(
    tree_base,              # modelo ya entrenado
    features_valid_scaled,  # X_valid → features de validación escalado
    target_valid,           # y_valid → etiquetas reales
    nombre_modelo="DecisionTreeClassifier (base)" # Nombre de identifiacion del modelo 
)
resultados.append(metrics_baseline)


#=========================
# DecisionTreeClassifier
#=========================
 

# Modelo + Mejores Hiperparametros 
model_rtree = DecisionTreeClassifier( 
    min_samples_leaf=best_leafs, # Obtenidos del punto 9.1
    max_depth=best_depth_tree,  # Obtenidos del punto 9.1
    random_state=RANDOM_STATE  # Definido en el punto 1.2
)
model_rtree.fit(features_upsampled, target_upsampled)

metrics_rtree = evaluar_modelo(
    model_rtree, # modelo ya entrenado
    features_valid_scaled,# X_valid → features de validación escalado
    target_valid, # y_valid → etiquetas reales
    nombre_modelo="DecisionTreeClassifier" # Nombre de identifiación del modelo
)
resultados.append(metrics_rtree)


#=========================
# RandomForestClassifier
#=========================

# Modelo + Mejores Hiperparametros 

model_rfc = RandomForestClassifier(
    random_state=RANDOM_STATE,  # Definido en el punto 1.2
    n_estimators=best_estimators_rf, # Obtenidos del punto 9.2
    max_depth=best_depth_rf
)
model_rfc.fit(features_upsampled, target_upsampled)

metrics_rfc = evaluar_modelo(
    model_rfc, # modelo ya entrenado
    features_valid_scaled,# X_valid → features de validación escalado
    target_valid, # y_valid → etiquetas reales
    nombre_modelo="RandomForestClassifier" # Nombre de identifiación del modelo
)
resultados.append(metrics_rfc)


#=========================
# LogisticRegression 
#=========================

# Modelo (normal)
model_lgr = LogisticRegression(
    random_state=RANDOM_STATE, # Definido en el punto 1.2
    solver="liblinear"

)
model_lgr.fit(features_upsampled, target_upsampled)

metrics_lgr = evaluar_modelo(
    model_lgr, # modelo ya entrenado
    features_valid_scaled,# X_valid → features de validación escalado
    target_valid, # y_valid → etiquetas reales
    nombre_modelo="LogisticRegression" # Nombre de identifiación del modelo
)
resultados.append(metrics_lgr)

# Juntamos todos los resultados de los modelos en una tabla
metrics_models = pd.concat(resultados, ignore_index=True)
# Ordenamos por F1 de forma decendente dejando los mejores resultados de f1 arriba
metrics_df_fmt = metrics_models.sort_values("F1", ascending=False).reset_index(drop=True)




### 11. 📝 Resultados y análisis

**Observaciones generales:**

- El objetivo del proyecto era alcanzar **F1 ≥ 0.59**.
- El modelo con **mayor F1** es: `RandomForestClassifier` con **F1 ≈ 0.62**.

---



In [69]:
display(metrics_df_fmt)

,modelo,Recall,F1,Curva PR,AUC-ROC,Precision,TP,FP,TN,FN
0,RandomForestClassifier,0.648536,0.621242,0.694817,0.871827,0.596154,310,210,1812,168
1,DecisionTreeClassifier,0.725941,0.598276,0.672317,0.861913,0.508798,347,335,1687,131
2,DecisionTreeClassifier (base),0.458159,0.517730,0.427579,0.746469,0.595109,219,149,1873,259
3,LogisticRegression,0.600418,0.493976,0.446312,0.778951,0.419591,287,397,1625,191


#### 11.2 🔍 Análisis del modelo seleccionado: RandomForestClassifier

**Métricas clave del RandomForestClassifier:**

- **Recall:** 0.6485  
  - Interpretación: de todos los clientes que realmente abandonan el banco, el modelo detecta aproximadamente el **65%**.  
  - TP (Verdaderos Positivos) = 310, FN (Falsos negativos)= 168.  
  - Es importante mantener un recall alto, ya que podemos identificar personas que se van del banco

- **Precision:** 0.5962  
  - Interpretación: de todos los clientes que el modelo marca como “se va”, alrededor del **60%** realmente se va.  
  - FP = 210.  
  - Al tener una precisión alta podemos hacer una campaña de retención mucho más barta, ya que realmente estaríamos llegando a las personas que realmente se van. De otra manera estamos desperdiciando presupuesto un 40 % de presupuesto en personas que pensamos que se van cuando realmente no se van del banco.

- **F1-score:** 0.6212  
  - Es la métrica que combina Precision y Recall, y es la que usamos como criterio principal.  
  - El modelo cumple el objetivo del proyecto (**F1 ≥ 0.59**).  
  - El modelo `RandomForestClassifier` considero que es el mejor modelo para este problema, ya que alcanzo mayor F1 comparación del resto de modelos de clasificación esto nos da un buen balance entre Precision y Recall.

- **AUC-ROC:** 0.8718  
  - Indica una muy buena capacidad de separación entre clientes que se quedan y clientes que se van.  
  - Tener un AUC-ROC cerca a 1 ayuda a organizar a los clientes con riesgo de que dejen el banco.

- **Curva PR (Average Precision):** 0.6948  
  - Refuerza que el modelo mantiene una buena relación entre Precision y Recall en diferentes umbrales.  
  - En promedio, a lo largo de distintos umbrales, el modelo mantiene una precisión bastante buena mientras aumenta el recall

#### 11.3 ⚖️ Comparación con otros modelos

- **DecisionTreeClassifier (Mejorado):**
  - Tiene un **Recall más alto (≈ 0.73)** que el RandomForest, pero una **Precision menor (≈ 0.51)**.
  - Esto significa que detecta más clientes que se van, pero también genera muchos más falsos positivos.
  - **F1 = 0.5983**, menor que el RandomForest.
  - Un mejor equilibrio nos ayuda a no incluir tantos clientes que realmente no se van (Falsos Positivos), maximisando nuestros esfuerzos en retencion de clientes.

- **DecisionTree (base):**
  - Serve como **modelo base** con F1 ≈ 0.52 y AUC-ROC ≈ 0.75.
  - Tiene menos recall (≈ 0.46) y deja escapar muchos clientes que abandonan (FN = 259).
  - Este modelo se toma como base inicial para analizar el comportamiento de los datos con un enfoque sencillo y sin optimización de hiperparámetros. Esto nos permite comparar de forma consistente distintos modelos sobre los mismos datos y elegir aquellos que ofrezcan el mejor desempeño.

- **LogisticRegression:**
  - Muestra un **AUC-ROC decente (≈ 0.78)**, pero F1 más bajo (≈ 0.49).
  - Su Precision es la más baja (≈ 0.42), generando muchos falsos positivos (FP = 397).
  - Este modelo no lo usamos ya que aunque tiene un AUC-ROC alto al comparar el F1 resulta mucho mas bajo que los otros modelos incluso mas bajo que el modelo base.


**Conclusión:**  
El **RandomForestClassifier** es el que mejor equilibra Precision, Recall y F1.  
Además tiene un AUC-ROC alto, así que es el modelo más sólido para este problema.


In [70]:
importancia = pd.DataFrame({
    "feature": features.columns,
    "importance": model_rfc.feature_importances_
}).sort_values("importance", ascending=False).reset_index(drop=True)

display(importancia)

,feature,importance
0,Age,0.331454
1,NumOfProducts,0.220817
2,Balance,0.107224
3,EstimatedSalary,0.074737
4,CreditScore,0.070907
5,IsActiveMember,0.056697
6,Geography_Germany,0.048976
7,Tenure,0.039229
8,Gender_Male,0.026373
9,Geography_Spain,0.008883


**Interpretación:**

- Las características **más influyentes** en la predicción de churn son:
  - **Age**
  - **NumOfProducts**
  - **Balance**
  - **EstimatedSalary**
  - **CreditScore**
- Esto sugiere que el riesgo de churn está muy relacionado con:
  - La **edad** del cliente,
  - La **cantidad de productos contratados**,
  - Y su **perfil financiero** (saldo, salario estimado, score de crédito).
- La variable `TenureWasMissing` tiene una importancia muy baja, lo que indica que:
  - Marcar los nulos fue útil para probar la hipótesis, pero no resultó ser un factor clave en la predicción

---




#### 11.5 🧠 Resumen del análisis

- Modelo final: **RandomForestClassifier**  
- F1 ≈ 0.62 (cumple la meta)  
- AUC-ROC ≈ 0.87 (muy buena separación)  
- Buen equilibrio entre detectar clientes en riesgo y no generar demasiados falsos positivos

**Variables más importantes:**
- Age  
- NumOfProducts  
- Balance  
- EstimatedSalary  
- CreditScore  

Estas variables reflejan el perfil financiero del cliente y qué tanto usa los productos del banco.

`TenureWasMissing` casi no aporta, así que no es una variable clave.

**En resumen:**  
El modelo ayuda a identificar clientes con alta probabilidad de irse para enfocar campañas de retención en ellos.  
Como siguiente paso, generaría una lista de clientes ordenados por riesgo y comenzaría acciones específicas para retenerlos.
